<a href="https://colab.research.google.com/github/Tsung-Hung/dummy-git/blob/master/%E6%AD%A1%E8%BF%8E%E4%BD%BF%E7%94%A8_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import torch 
import torch.nn as nn

# LayerNorm replace by BatchNorm

In [181]:
batch, sentence_length, embedding_dim = 2, 3, 5
embedding = torch.randn(batch, sentence_length, embedding_dim)
layer_norm = nn.LayerNorm(embedding_dim)
# Activate module
origin = layer_norm(embedding)
origin

tensor([[[ 1.0688, -0.9698, -1.2783,  1.1317,  0.0476],
         [ 0.0104,  1.0410, -0.4688, -1.6192,  1.0366],
         [ 1.1098,  0.4842, -0.3919, -1.7537,  0.5516]],

        [[-0.0934,  0.0651, -0.0703,  1.6269, -1.5282],
         [-0.4147,  0.0798,  1.5351, -1.5340,  0.3338],
         [ 0.3731, -1.6007,  0.6283, -0.6288,  1.2282]]],
       grad_fn=<NativeLayerNormBackward0>)

In [182]:
def Layer_norm(x):
    '''
    BatchNorm2d
    '''
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim, 1)
    layer_norm = nn.BatchNorm2d(x.shape[1], track_running_stats=False, affine=True)
    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

test = Layer_norm(embedding)
test

tensor([[[ 1.0688, -0.9698, -1.2783,  1.1317,  0.0476],
         [ 0.0104,  1.0410, -0.4688, -1.6192,  1.0366],
         [ 1.1098,  0.4842, -0.3919, -1.7537,  0.5516]],

        [[-0.0934,  0.0651, -0.0703,  1.6269, -1.5282],
         [-0.4147,  0.0798,  1.5351, -1.5340,  0.3338],
         [ 0.3731, -1.6007,  0.6283, -0.6288,  1.2282]]],
       grad_fn=<ReshapeAliasBackward0>)

In [183]:
def Layer_norm2(x):
    '''
    BatchNorm1d
    '''
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim)
    layer_norm = nn.BatchNorm1d(x.shape[1], track_running_stats=False, affine=True)
    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

test = Layer_norm2(embedding)
test

tensor([[[ 1.0688, -0.9698, -1.2783,  1.1317,  0.0476],
         [ 0.0104,  1.0410, -0.4688, -1.6192,  1.0366],
         [ 1.1098,  0.4842, -0.3919, -1.7537,  0.5516]],

        [[-0.0934,  0.0651, -0.0703,  1.6269, -1.5282],
         [-0.4147,  0.0798,  1.5351, -1.5340,  0.3338],
         [ 0.3731, -1.6007,  0.6283, -0.6288,  1.2282]]],
       grad_fn=<ReshapeAliasBackward0>)

# matmul 4D tensor

In [60]:
a1 = torch.rand(2,2,4,8)
a2 = torch.rand(2,2,6,8)

result = torch.matmul(a1, a2.transpose(-2,-1))
print('Shape : ', result.shape)
result

Shape :  torch.Size([2, 2, 4, 6])


tensor([[[[2.3131, 2.5684, 2.7268, 1.6015, 1.6414, 2.1339],
          [1.5039, 2.0396, 2.2715, 1.4535, 1.3327, 1.5806],
          [1.7024, 1.6837, 2.2664, 1.4777, 1.4518, 1.2221],
          [1.6095, 1.9949, 1.9517, 1.2658, 1.1603, 1.4011]],

         [[3.3467, 2.1018, 2.2768, 2.0836, 2.4307, 2.1606],
          [2.2287, 1.6298, 1.3702, 1.6120, 2.0341, 1.5944],
          [3.2428, 2.4924, 1.7053, 2.0370, 2.7048, 2.6245],
          [2.7496, 2.1988, 1.6442, 1.5780, 2.1083, 2.0812]]],


        [[[1.7565, 2.7733, 1.7162, 1.7396, 2.0423, 1.9646],
          [2.6238, 3.9487, 2.4292, 3.1959, 2.7582, 2.2436],
          [2.0262, 3.5778, 2.3311, 3.1846, 2.5255, 1.9627],
          [1.8414, 2.8815, 1.6750, 2.1518, 2.0514, 2.0574]],

         [[2.1957, 1.6194, 1.1019, 1.7354, 1.7786, 2.2907],
          [2.8351, 2.3687, 1.4348, 2.3595, 2.2277, 3.3091],
          [2.5764, 1.9971, 1.3812, 2.4410, 1.8875, 2.8381],
          [3.9394, 2.8348, 1.9002, 3.0424, 2.7458, 4.1299]]]])

In [61]:
def new_matmul1(a1,a2):

    k = a1.shape[0]*a1.shape[1]    
    mul_list = []
    for i in range(a1.shape[0]):
        for j in range(a1.shape[1]):
            
            mul = torch.matmul(a1[i,j,:],a2[i,j,:])                
            mul_list.append(mul)

    combine = torch.stack(mul_list)  
    combine = combine.view(a1.shape[0],a1.shape[1],combine.shape[1],combine.shape[2])
    
    return combine

result1 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result1.shape)
print(result1)

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.3131, 2.5684, 2.7268, 1.6015, 1.6414, 2.1339],
          [1.5039, 2.0396, 2.2715, 1.4535, 1.3327, 1.5806],
          [1.7024, 1.6837, 2.2664, 1.4777, 1.4518, 1.2221],
          [1.6095, 1.9949, 1.9517, 1.2658, 1.1603, 1.4011]],

         [[3.3467, 2.1018, 2.2768, 2.0836, 2.4307, 2.1606],
          [2.2287, 1.6298, 1.3702, 1.6120, 2.0341, 1.5944],
          [3.2428, 2.4924, 1.7053, 2.0370, 2.7048, 2.6245],
          [2.7496, 2.1988, 1.6442, 1.5780, 2.1083, 2.0812]]],


        [[[1.7565, 2.7733, 1.7162, 1.7396, 2.0423, 1.9646],
          [2.6238, 3.9487, 2.4292, 3.1959, 2.7582, 2.2436],
          [2.0262, 3.5778, 2.3311, 3.1846, 2.5255, 1.9627],
          [1.8414, 2.8815, 1.6750, 2.1518, 2.0514, 2.0574]],

         [[2.1957, 1.6194, 1.1019, 1.7354, 1.7786, 2.2907],
          [2.8351, 2.3687, 1.4348, 2.3595, 2.2277, 3.3091],
          [2.5764, 1.9971, 1.3812, 2.4410, 1.8875, 2.8381],
          [3.9394, 2.8348, 1.9002, 3.0424, 2.7458, 4.1299]

In [62]:
def new_matmul2(a1, a2):

    combine = torch.stack([torch.matmul(a1[i][j], a2[i][j]) for i in range(a1.shape[0]) for j in range(a1.shape[1])], dim=0)        
    return combine.reshape(a1.shape[0], a1.shape[1], a1.shape[2], a2.shape[3])

result2 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result2.shape)
print(result2)
print(torch.all(result==result2))

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.3131, 2.5684, 2.7268, 1.6015, 1.6414, 2.1339],
          [1.5039, 2.0396, 2.2715, 1.4535, 1.3327, 1.5806],
          [1.7024, 1.6837, 2.2664, 1.4777, 1.4518, 1.2221],
          [1.6095, 1.9949, 1.9517, 1.2658, 1.1603, 1.4011]],

         [[3.3467, 2.1018, 2.2768, 2.0836, 2.4307, 2.1606],
          [2.2287, 1.6298, 1.3702, 1.6120, 2.0341, 1.5944],
          [3.2428, 2.4924, 1.7053, 2.0370, 2.7048, 2.6245],
          [2.7496, 2.1988, 1.6442, 1.5780, 2.1083, 2.0812]]],


        [[[1.7565, 2.7733, 1.7162, 1.7396, 2.0423, 1.9646],
          [2.6238, 3.9487, 2.4292, 3.1959, 2.7582, 2.2436],
          [2.0262, 3.5778, 2.3311, 3.1846, 2.5255, 1.9627],
          [1.8414, 2.8815, 1.6750, 2.1518, 2.0514, 2.0574]],

         [[2.1957, 1.6194, 1.1019, 1.7354, 1.7786, 2.2907],
          [2.8351, 2.3687, 1.4348, 2.3595, 2.2277, 3.3091],
          [2.5764, 1.9971, 1.3812, 2.4410, 1.8875, 2.8381],
          [3.9394, 2.8348, 1.9002, 3.0424, 2.7458, 4.1299]

In [168]:
def new_matmul3(a1, a2):
  '''
  a1 = (2,2,4,8)
  a2 = (2,2,6,8)
  torch(a2, a1.T)
  '''
  B1, N1, H1, T1 = a1.shape
  B2, N2, T2, H2 = a2.shape
  a1 = a1.reshape(B1 * N1 * H1, T1)
  #print('a1 : ', a1.shape)
  a2 = a2.permute(2,0,1,3)
  a2 = a2.reshape(T2, B2 * N2 * H2)
  #print('a2 : ', a2.shape)
  result = torch.matmul(a1, a2)
  #result = torch.stack([result[i*H1:(i+1)*H1 ,i*H2:(i+1)*H2] for i in range(B1*N1)])
  #result = result.reshape(B1,N1,H1,H2)
  return result

result3 = new_matmul3(a1, a2.transpose(-2,-1))
print('Shape : ', result3.shape)
print(result3)

Shape :  torch.Size([60000, 4000])
tensor([[4.0756, 4.4000, 2.9250,  ..., 3.9321, 3.1452, 2.9357],
        [4.1753, 5.8372, 4.8736,  ..., 3.9643, 4.0882, 4.4215],
        [3.3328, 3.6199, 3.2783,  ..., 3.3327, 3.4274, 3.7227],
        ...,
        [3.7709, 4.7809, 3.8499,  ..., 3.6629, 4.3153, 3.3123],
        [2.5379, 3.2096, 2.3907,  ..., 2.9364, 3.2187, 2.7043],
        [3.3499, 5.3955, 3.9302,  ..., 3.6605, 4.5480, 4.3740]],
       device='cuda:0')


In [69]:
def new_matmul4(a1, a2):
  '''
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8)
  a1_matrix = (16,32)
  a2_matrix = (32,24)
  '''
  #a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
  #a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])
  #identity = torch.cat([torch.eye(a2.shape[3]) for _ in range(a2.shape[0]*a2.shape[1])]).cuda()
  a = torch.ones(a2.shape[0]*a2.shape[1],a2.shape[3])
  identity2 = torch.diag_embed(a).reshape(a2.shape[0]*a2.shape[1]*a2.shape[3], a2.shape[3])
  result = torch.matmul(a1_matrix, a2_matrix)
  result = torch.matmul(result, identity2)
  return result

result4 = new_matmul4(a1,a2.transpose(-2,-1))
print('Shape : ', result4.shape)
print(result4)

Shape :  torch.Size([16, 6])
tensor([[2.3131, 2.5684, 2.7268, 1.6015, 1.6414, 2.1339],
        [1.5039, 2.0396, 2.2715, 1.4535, 1.3327, 1.5806],
        [1.7024, 1.6837, 2.2664, 1.4777, 1.4518, 1.2221],
        [1.6095, 1.9949, 1.9517, 1.2658, 1.1603, 1.4011],
        [3.3467, 2.1018, 2.2768, 2.0836, 2.4307, 2.1606],
        [2.2287, 1.6298, 1.3702, 1.6120, 2.0341, 1.5944],
        [3.2428, 2.4924, 1.7053, 2.0370, 2.7048, 2.6245],
        [2.7496, 2.1988, 1.6442, 1.5780, 2.1083, 2.0812],
        [1.7565, 2.7733, 1.7162, 1.7396, 2.0423, 1.9646],
        [2.6238, 3.9487, 2.4292, 3.1959, 2.7582, 2.2436],
        [2.0262, 3.5778, 2.3311, 3.1846, 2.5255, 1.9627],
        [1.8414, 2.8815, 1.6750, 2.1518, 2.0514, 2.0574],
        [2.1957, 1.6194, 1.1019, 1.7354, 1.7786, 2.2907],
        [2.8351, 2.3687, 1.4348, 2.3595, 2.2277, 3.3091],
        [2.5764, 1.9971, 1.3812, 2.4410, 1.8875, 2.8381],
        [3.9394, 2.8348, 1.9002, 3.0424, 2.7458, 4.1299]])


In [ ]:
def new_matmul_final(a1, a2):
  '''
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8)
  a1_matrix = (16,32)
  a2_matrix = (32,24)
  '''
  a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
  a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])
  #identity = torch.cat([torch.eye(a2.shape[3]) for _ in range(a2.shape[0]*a2.shape[1])]).cuda()
  a = torch.ones(a2.shape[0]*a2.shape[1],a2.shape[3])
  #identity2 = torch.diag_embed(a).reshape(a2.shape[0]*a2.shape[1]*a2.shape[3], a2.shape[3]).cuda()
  result = torch.matmul(a1_matrix, a2_matrix)
  #result = torch.matmul(result, identity2)
  return result

#result4 = new_matmul4(a1,a2.transpose(-2,-1))
#print('Shape : ', result4.shape)
#print(result4)

In [157]:
a2 = torch.rand(1,2,3,4)
a2

tensor([[[[0.7210, 0.2294, 0.4558, 0.0930],
          [0.2017, 0.0607, 0.4425, 0.3461],
          [0.5573, 0.3538, 0.4102, 0.7203]],

         [[0.8404, 0.7805, 0.1481, 0.0404],
          [0.7765, 0.7399, 0.3273, 0.6756],
          [0.1364, 0.3438, 0.1867, 0.5070]]]])

In [158]:

B2, N2, T2, H2 = a2.shape
a2 = a2.reshape( B2 * N2* T2, H2)
print(a2.shape)
one = torch.ones(B2*N2, H2)
a2_I = torch.diag_embed(one).permute(1,0,2).reshape(H2, B2*N2*H2)
a2_m = torch.matmul(a2, a2_I)
a2_m

torch.Size([6, 4])


tensor([[0.7210, 0.2294, 0.4558, 0.0930, 0.7210, 0.2294, 0.4558, 0.0930],
        [0.2017, 0.0607, 0.4425, 0.3461, 0.2017, 0.0607, 0.4425, 0.3461],
        [0.5573, 0.3538, 0.4102, 0.7203, 0.5573, 0.3538, 0.4102, 0.7203],
        [0.8404, 0.7805, 0.1481, 0.0404, 0.8404, 0.7805, 0.1481, 0.0404],
        [0.7765, 0.7399, 0.3273, 0.6756, 0.7765, 0.7399, 0.3273, 0.6756],
        [0.1364, 0.3438, 0.1867, 0.5070, 0.1364, 0.3438, 0.1867, 0.5070]])

In [155]:
one = torch.ones(B2*N2, H2)
a2_I = torch.diag_embed(one).permute(1,0,2).reshape(H2, B2*N2*H2)
a2_I

tensor([[1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1.]])

In [100]:
#test = torch.eye(a2.shape[1])
one = torch.ones(2,4)
a2_I = torch.diag_embed(one)
a2_I
#test = torch.matmul(a2,test)
#test

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]])

In [175]:
a1 = torch.rand(1,4,15000,16).cuda()
a2 = torch.rand(1,4,1000,16).cuda()

%timeit -n 100 -r 7 torch.matmul(a1, a2.transpose(-2,-1))

25.4 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [176]:
%timeit -n 100 -r 7 new_matmul1(a1, a2.transpose(-2,-1))

3.99 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [177]:
%timeit -n 100 -r 7 new_matmul2(a1, a2.transpose(-2,-1))

3.57 ms ± 46.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [178]:
%timeit -n 200 -r 7 new_matmul3(a1, a2.transpose(-2,-1))

The slowest run took 8.39 times longer than the fastest. This could mean that an intermediate result is being cached.
2.01 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 200 loops each)


In [36]:
%timeit -n 100 -r 7 new_matmul4(a1, a2.transpose(-2,-1))

11.5 ms ± 7.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [37]:
a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])

%timeit -n 100 -r 7 torch.matmul(a1_matrix, a2_matrix.transpose(-2,-1))

746 µs ± 36.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [97]:
from numpy import matrix
A = matrix([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0],[0,0,1]])
A2 = A.I
B = matrix([[1,3,2],[6,1,2],[3,1,1],[1,4,1],[1,1,2],[3,2,1]])
B * A2 * 2

matrix([[1., 3., 2., 1., 3., 2.],
        [6., 1., 2., 6., 1., 2.],
        [3., 1., 1., 3., 1., 1.],
        [1., 4., 1., 1., 4., 1.],
        [1., 1., 2., 1., 1., 2.],
        [3., 2., 1., 3., 2., 1.]])

In [161]:
AB = matrix([[1,3,2,0,0,0],[6,1,2,0,0,0],[3,1,1,0,0,0],[0,0,0,1,4,1],[0,0,0,1,1,2],[0,0,0,3,2,1]])
I1 = matrix([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0],[0,0,1]])
I2 = matrix([[1,0,0,1,0,0],[0,1,0,0,1,0],[0,0,1,0,0,1]])
res = AB*I1
res

matrix([[1, 3, 2],
        [6, 1, 2],
        [3, 1, 1],
        [1, 4, 1],
        [1, 1, 2],
        [3, 2, 1]])